In [305]:
import pandas as pd
import numpy as np
import os
import pyreadr
import json
from datetime import date
import datetime
from os import listdir
import uuid
from tqdm import tqdm
from datetime import datetime
from datetime import timedelta

In [47]:
import warnings
warnings.filterwarnings("ignore")

## Functions

In [ ]:
def add_missing_timesteps(temp):
    missing_rows = []
    for i in list(range(-16, 16)):
        if i not in temp.days_since_factcheck.unique():
            row =[np.nan, 
             np.nan,
             np.nan,
             np.nan,
             np.nan,
             np.nan,
             np.nan,
             np.nan,
             np.nan,
             np.nan]
            timestep = list(temp.date_factcheck)[0] + timedelta(i)
            
            row.append(timestep)
            row.append(list(temp.id_desinformacion)[0])
            row.append(list(temp.id_placebo)[0])
            row.append(list(temp.date_desinformacion)[0])
            row.append(list(temp.date_publicacion)[0])
            row.append((timestep - list(temp.date_publicacion)[0]).days)
            row.append(list(temp.date_factcheck)[0])
            row.append(i)
            row.append(list(temp.id_post_placebo)[0])
            row.append(list(temp.id_factcheck)[0])
            row.append(list(temp.link_placebo)[0])
            row.append(list(temp.days_since_misinformation_relative_fc)[0])
            m_row = pd.DataFrame([row], columns = list(temp.columns))
            missing_rows.append(m_row)
    temp_clean = pd.concat([temp, pd.concat(missing_rows)])
    temp_clean = temp_clean[temp_clean['days_since_publication'] >= 0]
    return(temp_clean)

In [26]:
def load_regression_panel():
    result = pyreadr.read_r('../../../data/5-analysis/1-input_data/misinformation/data_reg_misinformation.Rda')
    panel = result["input_reg"]
    panel_reg = panel[['id_desinformacion', 'type_misinformation','link_desinformacion', 'label_desinformacion']]
    panel_reg = panel_reg.drop_duplicates().reset_index().drop('index', axis = 1)
    return(panel_reg)

In [410]:
def load_regression_data():
    result = pyreadr.read_r('../../../data/5-analysis/1-input_data/misinformation/data_reg_misinformation.Rda')
    panel = result["input_reg"]
    return(panel)

In [332]:
def load_social_posts():
    posts_social = []
    for i in listdir('../../../data/3-synthetic_controls/social/5-clean_controls/'):
        try:
            posts_social.append(pd.read_json('../../../data/3-synthetic_controls/social/5-clean_controls/' + i))
        except:
            continue
    return(pd.concat(posts_social))

In [329]:
def load_regression_panel_social():
    result = pyreadr.read_r('../../../data/5-analysis/1-input_data/misinformation/data_reg_misinformation.Rda')
    panel = result["input_reg"]
    panel_reg = panel[['id_desinformacion', 'type_misinformation','link_desinformacion', 'label_desinformacion', 'date_desinformacion']]
    panel_reg = panel_reg.drop_duplicates().reset_index().drop('index', axis = 1)
    panel_reg = panel_reg[panel_reg['type_misinformation'] == 'social_media']
    return(panel_reg)

In [276]:
def load_posts_media():
    path = '/Users/cblanesg/misinformation_socialmedia/data/3-synthetic_controls/newspapers/3-pre_controls/posts_crowdtangle.json'
    with open(path, 'r') as f:
        data = json.load(f)

    path = '/Users/cblanesg/misinformation_socialmedia/data/3-synthetic_controls/newspapers/3-pre_controls/posts_crowdtangle_true.json'
    with open(path, 'r') as f:
        data_true = json.load(f)
        
    df_posts = pd.concat([pd.DataFrame(data), 
                     pd.DataFrame(data_true)])
    return(df_posts)

## Apply Functions

In [297]:
panel_reg = load_regression_panel()

In [298]:
id_placebo = pd.read_excel('../../../data/3-synthetic_controls/newspapers/0-ids/media_ids.xlsx')

In [299]:
len(panel_reg.id_desinformacion.unique())

605

In [300]:
id_placebo_media['date_desinformacion'] = id_placebo_media.apply(lambda x: datetime.date(x['year'], 
                                                                                         x['month'], 
                                                                                        x['days']).strftime('%Y-%m-%d'), axis = 1)


In [301]:
id_newspapers = panel_reg[panel_reg['type_misinformation'] == 'media_outlets']

In [302]:
id_newspapers = pd.merge(left= id_newspapers, 
        right = id_placebo_media[['id_desinformacion', 'date_desinformacion', 'media_outlet']], 
        on = 'id_desinformacion', 
        how = 'left')

## Placebo Newspapers

In [219]:
id_placebo_media = id_placebo[id_placebo['id_desinformacion'].isin(panel_reg.id_desinformacion)]

In [220]:
id_placebo_media['date_desinformacion'] = id_placebo_media.apply(lambda x: datetime.date(x['year'], 
                                                                                         x['month'], 
                                                                                        x['days']), axis = 1)

## Status of Media Outlets

In [277]:
posts_facebook = load_posts_media()

In [280]:
id_newspapers = id_newspapers.reset_index()

In [315]:
id_newspapers

,id_desinformacion,type_misinformation,link_desinformacion,label_desinformacion,date_desinformacion,media_outlet
0,01839a6d-1c36-3b22-a0c2-2ed9a2633a6e,media_outlets,https://colombianoindignado.com/nino-respondio...,true,2019-03-10,colombianoindignado
1,02400198-d2d3-38c7-b226-eb98854a750e,media_outlets,https://www.dw.com/es/informe-gobierno-alem%C3...,true,2019-03-27,dw
2,02ae185c-282c-41bc-97f0-7c0503aca187,media_outlets,https://albaciudad.org/2020/04/persecucion-y-r...,fake,2020-04-11,albaciudad
3,02e87747-3140-3eaf-94b1-dde6ad6abc5c,media_outlets,https://colombianoindignado.com/new-york-times...,true,2019-05-18,colombianoindignado
4,032787bb-a1c6-300b-98de-418c660c7f45,media_outlets,https://www.elespectador.com/noticias/el-mundo...,true,2019-02-21,elespectador
...,...,...,...,...,...,...
501,fcc2bf94-a191-33ee-8310-50e5447d21ea,media_outlets,https://diadelsur.com/el-uso-prolongado-del-ta...,fake,2020-05-03,diadelsur
502,fd73d687-5922-33a3-bc97-1b40d48d4b84,media_outlets,https://www.las2orillas.co/los-5-magistrados-d...,misleading,2019-05-15,las2orillas
503,fd968c77-e60d-3dcf-aa12-f42a2efcdc58,media_outlets,https://www.news24.com/news24/columnists/guest...,fake,2020-08-21,news24
504,fe0e2aaf-233c-4726-bbb4-c1bb9024fa15,media_outlets,https://codigooculto.com/2019/12/cientos-de-av...,fake,2019-12-30,codigooculto


In [316]:
status_newspapers = {}

In [317]:
### Check status of placebo data
for i in tqdm(range(0, len(id_newspapers))):
    id_desinformacion = id_newspapers.loc[i]['id_desinformacion']
    date_desinformacion = id_newspapers.loc[i]['date_desinformacion']
    media_outlet = id_newspapers.loc[i]['media_outlet']
    
    
    if id_desinformacion in list(posts_facebook.id_desinformacion):
        posts_placebo = posts_facebook[posts_facebook['id_desinformacion'] == id_desinformacion]
        
        posts_placebo = posts_placebo[posts_placebo['dummy_text_article'] == 1]
        if len(posts_placebo) == 0:
            status_newspapers[id_desinformacion] = 'yes posts, no articles'
        else:
            posts_placebo = posts_placebo[posts_placebo['date_control'] == date_desinformacion]
            if len(posts_placebo) == 0:
                status_newspapers[id_desinformacion] = 'yes posts with articles, not same date'
            else:
                status_newspapers[id_desinformacion] = 'with placebo'

    elif not pd.isna(media_outlet) and media_outlet + '.json' in listdir(path_scrapes):
        scrapes = pd.read_json(path_scrapes + id_newspapers.loc[i]['media_outlet'] + '.json')
        scrapes_same_day = scrapes[scrapes['date'] == date_desinformation]
        
        if len(scrapes_same_day) > 0:
            status_newspapers[id_desinformacion] = 'with placebo'
            #scrapes_same_day['id_desinformacion'] = missing_newspapers.loc[i]['id_desinformacion']
            #scrapes_same_day = scrapes_same_day.drop_duplicates('id_desinformacion', keep = 'first')
            #matches_scrapes.append(scrapes_same_day)
        else:
            status_newspapers[id_desinformacion] = 'yes articles, not same day'
    
    else:
        status_newspapers[id_desinformacion] = 'no articles, no posts'
        

100%|██████████| 506/506 [00:10<00:00, 48.43it/s]


In [320]:
status_media_outlets = pd.DataFrame(list(status_newspapers.items()),columns = ['id_desinformacion','status']) 

In [323]:
status_media_outlets.groupby(['status']).count()

,id_desinformacion
status,
"no articles, no posts",232
with placebo,126
"yes articles, not same day",17
"yes posts with articles, not same date",58
"yes posts, no articles",63


## Status of Posts

In [330]:
panel_reg_social = load_regression_panel_social()

In [334]:
panel_reg_social = panel_reg_social.reset_index()

In [336]:
social_posts = load_social_posts()

In [339]:
status_social = {}

In [340]:
for i in tqdm(range(0, len(panel_reg_social))):
    id_desinformacion = panel_reg_social.loc[i]['id_desinformacion']
    date_desinformacion = panel_reg_social.loc[i]['date_desinformacion']
    
    if id_desinformacion in list(social_posts.id_desinformacion):
        posts_placebo = social_posts[social_posts['id_desinformacion'] == id_desinformacion]
        posts_placebo = posts_placebo[posts_placebo['desinformacion'] == 0]
        
        same_date = posts_placebo[posts_placebo['n_days_since_desinformation'] == 0]
        if len(same_date) == 0:
            status_social[id_desinformacion] = 'yes posts,not same day'
            
        else:
            history_posts = same_date[~same_date['history_control'].isnull()]
            
            if len(history_posts) == 0:
                status_social[id_desinformacion] = 'yes posts, not history'
            else:
                status_social[id_desinformacion] = 'with placebo'
    else:
        status_social[id_desinformacion] = 'no posts'
        

100%|██████████| 109/109 [00:01<00:00, 79.22it/s]


In [341]:
status_social_df = pd.DataFrame(list(status_social.items()),columns = ['id_desinformacion','status']) 

In [342]:
status_social_df.groupby(['status']).count()

,id_desinformacion
status,
no posts,31
with placebo,52
"yes posts, not history",18
"yes posts,not same day",8


## Obtain cases with Placebo

In [354]:
newspapers_placebo = []

In [355]:
for i in tqdm(range(0, len(id_newspapers))):
    id_desinformacion = id_newspapers.loc[i]['id_desinformacion']
    date_desinformacion = id_newspapers.loc[i]['date_desinformacion']
    media_outlet = id_newspapers.loc[i]['media_outlet']
    
    
    if id_desinformacion in list(posts_facebook.id_desinformacion):
        posts_placebo = posts_facebook[posts_facebook['id_desinformacion'] == id_desinformacion]
        
        posts_placebo = posts_placebo[posts_placebo['dummy_text_article'] == 1]
        if len(posts_placebo) == 0:
            #status_newspapers[id_desinformacion] = 'yes posts, no articles'
            pass
        else:
            posts_placebo = posts_placebo[posts_placebo['date_control'] == date_desinformacion]
            if len(posts_placebo) == 0:
                #status_newspapers[id_desinformacion] = 'yes posts with articles, not same date'
                pass
            else:
                #status_newspapers[id_desinformacion] = 'with placebo'
                temp = posts_placebo.drop_duplicates('id_desinformacion', keep = 'first')
                temp['id_placebo'] = temp['post_control'].apply(lambda x: str(uuid.uuid3(uuid.NAMESPACE_URL,x)))
                temp = temp[['id_desinformacion', 'id_placebo', 'date_desinformacion', 'link_control']].rename(columns = {'link_control':'link_placebo'})
                newspapers_placebo.append(temp)

    elif not pd.isna(media_outlet) and media_outlet + '.json' in listdir(path_scrapes):
        scrapes = pd.read_json(path_scrapes + id_newspapers.loc[i]['media_outlet'] + '.json')
        scrapes_same_day = scrapes[scrapes['date'] == date_desinformation]
        
        if len(scrapes_same_day) > 0:
            status_newspapers[id_desinformacion] = 'with placebo'
            scrapes_same_day['id_desinformacion'] = id_desinformacion
            scrapes_same_day = scrapes_same_day.drop_duplicates('id_desinformacion', keep = 'first')
            
            temp = scrapes_same_day[['id_desinformacion', 
                  'id_article', 
                 'date', 
                 'url_article']].rename(columns = {'id_article':'id_placebo', 
                                                                       'date':'date_desinformacion', 
                                                  'url_article':'link_placebo'})
            newspapers_placebo.append(temp)
        else:
            #status_newspapers[id_desinformacion] = 'yes articles, not same day'
            pass
    
    else:
        #status_newspapers[id_desinformacion] = 'no articles, no posts'
        pass

100%|██████████| 506/506 [00:11<00:00, 43.68it/s]


In [356]:
newspapers_placebo = pd.concat(newspapers_placebo)

In [358]:
newspapers_placebo['type_misinformacion'] = 'media_outlets'

## Social Media

In [361]:
social_placebo = []

In [362]:
for i in tqdm(range(0, len(panel_reg_social))):
    id_desinformacion = panel_reg_social.loc[i]['id_desinformacion']
    date_desinformacion = panel_reg_social.loc[i]['date_desinformacion']
    
    if id_desinformacion in list(social_posts.id_desinformacion):
        posts_placebo = social_posts[social_posts['id_desinformacion'] == id_desinformacion]
        posts_placebo = posts_placebo[posts_placebo['desinformacion'] == 0]
        
        same_date = posts_placebo[posts_placebo['n_days_since_desinformation'] == 0]
        if len(same_date) == 0:
            #status_social[id_desinformacion] = 'yes posts,not same day'
            pass
            
        else:
            history_posts = same_date[~same_date['history_control'].isnull()]
            
            if len(history_posts) == 0:
                #status_social[id_desinformacion] = 'yes posts, not history'
                pass
            else:
                temp = history_posts.drop_duplicates('id_desinformacion', keep = 'first')
                temp['id_placebo'] = temp['link_post'].apply(lambda x: str(uuid.uuid3(uuid.NAMESPACE_URL,x)))
                #status_social[id_desinformacion] = 'with placebo'
                temp = temp[['id_desinformacion', 
                      'id_placebo',
                      'date_post', 
                     'link_post', 
                     'history_control']].rename(columns = {'link_post':'link_placebo', 
                                                    'history_control':'history_placebo', 
                                                          'date_post':'date_desinformacion'})
                social_placebo.append(temp)
    else:
        #status_social[id_desinformacion] = 'no posts'
        pass

100%|██████████| 109/109 [00:01<00:00, 72.75it/s]


In [363]:
social_placebo = pd.concat(social_placebo)
social_placebo['type_desinformacion'] = 'social_media'

# Obtain all posts shared in Crowdtangle of placebo media outlets

In [372]:
import requests
api_dashboard = 'V6lMawCCbCP2rShqtU3TmYCY3Em1Osd8I2DLYxwr'

In [369]:
newspapers_placebo = newspapers_placebo.reset_index()

In [395]:
posts_newspapers = []

In [396]:
for i in tqdm(range(5, len(newspapers_placebo))):
    id_placebo = newspapers_placebo.loc[i]['id_placebo']
    id_desinformacion = newspapers_placebo.loc[i]['id_desinformacion']
    date_desinformacion = newspapers_placebo.loc[i]['date_desinformacion']
    link_placebo = newspapers_placebo.loc[i]['link_placebo']
    
    URL_BASE = "https://api.crowdtangle.com/links"
    PARAMS = {'link': i, 'count': 1000,'token': api_dashboard, 'platforms': 'facebook',
         'includeHistory':'true'}
    
    r = requests.get(url = URL_BASE, params=PARAMS)
    if len(r.content) == 0:
        pass
    else:
        data = r.json()
        out = data['result']['posts']

        if len(out) >= 1:
            df = pd.DataFrame(out)
            df['id_desinformacion'] = id_desinformacion
            df['id_placebo'] = id_placebo
            df['date_desinformacion'] = date_desinformacion
            df['link_placebo'] = link_placebo
            df['id_post_placebo'] = df['postUrl'].apply(lambda x: str(uuid.uuid3(uuid.NAMESPACE_URL,x)))
            df = df[['id_desinformacion', 
                    'id_placebo', 
                     'id_post_placebo',
                    'date_desinformacion', 
                    'link_placebo', 
                    'history']]
            posts_newspapers.append(df)
        else:
            pass

100%|██████████| 122/122 [23:03<00:00, 11.34s/it]


In [419]:
posts_newspapers_placebo = pd.concat(posts_newspapers)

In [420]:
posts_newspapers_placebo['type_desinformacion'] = 'media_outlets'
posts_newspapers_placebo = posts_newspapers_placebo.rename(columns = {'history':'history_placebo'})

In [399]:
social_placebo['id_post_placebo'] = social_placebo['id_placebo']
social_placebo = social_placebo[['id_desinformacion', 
                    'id_placebo', 
                     'id_post_placebo',
                    'date_desinformacion', 
                    'link_placebo', 
                    'history_placebo', 
               'type_desinformacion']]

## Merge Placebo + Media Outlets

In [400]:
placebo_data = pd.concat([social_placebo, 
                         posts_newspapers_placebo])

## Assign date of fact check

In [411]:
panel_data = load_regression_data()

In [412]:
panel_data.columns

Index(['date_timestep', 'id_desinformacion', 'date_desinformacion',
       'days_since_publication', 'date_factcheck', 'days_since_factcheck',
       'id_post_desinformacion', 'id_factcheck', 'link_desinformacion',
       'facebook_partnership_date', 'organizacion', 'pais', 'likes', 'shares',
       'comments', 'reactions', 'id_factcheck_post_misinf', 'approx_likes',
       'approx_shares', 'approx_comments', 'approx_reactions',
       'approx_interactions', 'growth_likes', 'growth_shares',
       'growth_reactions', 'growth_comments', 'growth_interactions',
       'type_misinformation', 'label_desinformacion', 'treatment',
       'days_since_poynter', 'poynter', 'label_desinformacion_pool',
       'factcheck_engagement', 'popular', 'as.factor(popular)',
       'days_since_misinformation_relative_fc'],
      dtype='object')

In [415]:
extra_data = panel_data[['id_desinformacion', 
          'id_factcheck', 
            'date_factcheck',
            'days_since_misinformation_relative_fc']].drop_duplicates()

In [417]:
placebo_data = pd.merge(left = placebo_data, 
        right = extra_data, 
        on = 'id_desinformacion', 
        how = 'left')

In [418]:
placebo_data

,id_desinformacion,id_placebo,id_post_placebo,date_desinformacion,link_placebo,history_placebo,type_desinformacion,id_factcheck,date_factcheck,days_since_misinformation_relative_fc
0,117a5f43-7cba-4b7d-b6e4-d0141baeeba4,52374bf6-4f70-3b12-9e4f-d50b4911f2e8,52374bf6-4f70-3b12-9e4f-d50b4911f2e8,2020-06-21,https://www.facebook.com/RadioKawsachunCoca/po...,"[{'actual': {'likeCount': 672, 'shareCount': 8...",social_media,1df63fdf-2335-3ca3-b366-8be0c196d37b,2020-06-24,3.0
1,14744415-8ce8-31d9-9835-31580110b342,bc0a3913-702f-3397-9c3b-d942c9ad02e4,bc0a3913-702f-3397-9c3b-d942c9ad02e4,2020-04-03,https://www.facebook.com/YoBancoAJorgeLanata/p...,"[{'actual': {'likeCount': 20, 'shareCount': 5,...",social_media,f6ddf69f-45d9-3666-8b39-224c4f9e7bd1,2020-06-10,15.0
2,15b5fd51-ca22-4ed4-971b-2549ca8f95d9,d3280acb-5c94-301c-bf25-bf67a4af0c6c,d3280acb-5c94-301c-bf25-bf67a4af0c6c,2020-07-14,https://www.facebook.com/1000manerasdeanulartu...,"[{'actual': {'likeCount': 121, 'shareCount': 9...",social_media,f1522e0d-ec24-3830-a2f6-194b6b45f0f2,2020-07-15,1.0
3,20b53772-747e-3366-a9ec-2a61d5d6a7d1,1c8ff57e-d1c1-38b6-a450-7fa75e0fc8ff,1c8ff57e-d1c1-38b6-a450-7fa75e0fc8ff,2018-12-12,https://www.facebook.com/LasAventurasDeSalinas...,"[{'actual': {'likeCount': 1805, 'shareCount': ...",social_media,0f707fd2-8b3b-384c-bedc-929b7ad3dfc2,2018-12-27,15.0
4,2420094d-a184-317d-8245-0af184aae06d,90cb5945-a220-3e7c-8e8c-6d5c980ba149,90cb5945-a220-3e7c-8e8c-6d5c980ba149,2020-04-03,https://www.facebook.com/jeshusf/posts/2070549...,"[{'actual': {'likeCount': 142, 'shareCount': 1...",social_media,788b8087-8075-3366-806f-68a44083b41e,2020-04-07,4.0
...,...,...,...,...,...,...,...,...,...,...
122047,ff5b98f7-7da4-3f5e-9b5b-c123c032defe,ea8777df-6af1-331d-a9e9-8ffae4a27171,1c062e45-c71a-39b6-b631-9d7ab8e43f74,2020-06-01,https://www.bluradio.com/economia/que-diferenc...,"[{'actual': {'likeCount': 0, 'shareCount': 0, ...",media_outlets,8b380cfa-acd4-35e2-95f0-49e48b2b7785,2020-06-03,2.0
122048,ff5b98f7-7da4-3f5e-9b5b-c123c032defe,ea8777df-6af1-331d-a9e9-8ffae4a27171,a426aeb9-5e30-3bdb-bfc9-84d0457be33d,2020-06-01,https://www.bluradio.com/economia/que-diferenc...,"[{'actual': {'likeCount': 0, 'shareCount': 0, ...",media_outlets,8b380cfa-acd4-35e2-95f0-49e48b2b7785,2020-06-03,2.0
122049,ff5b98f7-7da4-3f5e-9b5b-c123c032defe,ea8777df-6af1-331d-a9e9-8ffae4a27171,3fd5fe0d-5efe-3f72-9e00-825865fb7ae7,2020-06-01,https://www.bluradio.com/economia/que-diferenc...,"[{'actual': {'likeCount': 0, 'shareCount': 0, ...",media_outlets,8b380cfa-acd4-35e2-95f0-49e48b2b7785,2020-06-03,2.0
122050,ff5b98f7-7da4-3f5e-9b5b-c123c032defe,ea8777df-6af1-331d-a9e9-8ffae4a27171,ac57a206-522c-3370-9535-2a377000e1b9,2020-06-01,https://www.bluradio.com/economia/que-diferenc...,"[{'actual': {'likeCount': 1, 'shareCount': 0, ...",media_outlets,8b380cfa-acd4-35e2-95f0-49e48b2b7785,2020-06-03,2.0


## Make Panel

In [457]:
df_all = []

In [475]:
for i in tqdm(range(0, len(placebo_data))):
    post = placebo_data.loc[i]
    
    df_history = pd.DataFrame(post['history_placebo'])
    df_history['date_timestep'] = df_history['date'].apply(lambda x: datetime.strptime(x.split(' ')[0], '%Y-%m-%d').date())
    df_history = df_history.reset_index().drop('index', axis = 1)

    df_eng_post = []
    for h in range(0, len(df_history)):
        columns = list(df_history.loc[h]['actual'].keys())
        values = list(df_history.loc[h]['actual'].values())
        df_temp = pd.DataFrame([values], columns = columns)
        df_temp['date_timestep'] = df_history.loc[h]['date_timestep']
        df_temp['id_desinformacion'] = post['id_desinformacion']
        df_temp['id_placebo'] = post['id_placebo']
        df_temp = df_temp.drop_duplicates('date_timestep', keep = 'first')    
        df_temp['date_desinformacion'] = post['date_desinformacion']
        df_temp['date_publicacion'] = datetime.strptime(post['history_placebo'][len(post['history_placebo']) - 1]['date'].split(' ')[0],
                                                        '%Y-%m-%d').date()
        df_temp['days_since_publication'] = df_temp.apply(lambda x: (x['date_timestep'] - x['date_publicacion']).days, axis = 1)
        
        df_temp['date_factcheck'] = datetime.strptime(str(post['date_factcheck']), '%Y-%m-%d').date()
        df_temp['days_since_factcheck'] = df_temp.apply(lambda x: (x['date_timestep'] - x['date_factcheck']).days, axis = 1)
        df_temp['id_post_placebo'] = post['id_post_placebo']
        df_temp['id_factcheck'] = post['id_factcheck']
        df_temp['link_placebo'] = post['link_placebo']
        df_temp['days_since_misinformation_relative_fc'] = post['days_since_misinformation_relative_fc']
        df_eng_post.append(df_temp)
        
        temp = pd.concat(df_eng_post).drop_duplicates('days_since_publication', keep = 'first')
        panel = add_missing_timesteps(temp)
        df_all.append(panel)



100%|██████████| 122052/122052 [4:17:37<00:00,  7.90it/s]   


In [476]:
panel_placebo = pd.concat(df_all)

## Interpolate data

In [477]:
from tqdm import tqdm
tqdm.pandas()

In [478]:
placebo_temp_panel  = panel_placebo.sort_values(['id_post_placebo',
                                                 'days_since_publication']).reset_index()#.drop('rownames', axis = 1)

In [479]:
placebo_temp_panel['loveCount'] = pd.to_numeric(placebo_temp_panel.loveCount)
placebo_temp_panel['hahaCount'] = pd.to_numeric(placebo_temp_panel.hahaCount)
placebo_temp_panel['wowCount'] = pd.to_numeric(placebo_temp_panel.wowCount)
placebo_temp_panel['sadCount'] = pd.to_numeric(placebo_temp_panel.sadCount)
placebo_temp_panel['angryCount'] = pd.to_numeric(placebo_temp_panel.angryCount)
placebo_temp_panel['thankfulCount'] = pd.to_numeric(placebo_temp_panel.thankfulCount)
placebo_temp_panel['careCount'] = pd.to_numeric(placebo_temp_panel.careCount)

In [480]:
placebo_temp_panel['reactions'] = placebo_temp_panel.apply(lambda x: 
                                                           x['loveCount'] 
                                                           + x['hahaCount']
                                                           + x['wowCount']
                                                           + x['sadCount']
                                                           + x['angryCount'] 
                                                           + x['thankfulCount']
                                                           + x['careCount'], axis = 1)

In [481]:
placebo_temp_panel['likes'] = pd.to_numeric(placebo_temp_panel.likeCount)
placebo_temp_panel['shares'] = pd.to_numeric(placebo_temp_panel.shareCount)
placebo_temp_panel['comments'] = pd.to_numeric(placebo_temp_panel.commentCount)

In [483]:
panel_input = placebo_temp_panel[['date_timestep', 
                                  'id_desinformacion', 
                                  'id_placebo', 
                                  'date_publicacion',
                                  'date_desinformacion',
                                  'days_since_publication',
                                  'date_factcheck',
                                  'days_since_factcheck',
                                  'id_post_placebo',
                                  'id_factcheck',
                                  'link_placebo',
                                  'likes', 'shares', 'comments', 'reactions']]

In [484]:
panel_input = panel_input.sort_values(['id_post_placebo','days_since_publication'])

In [485]:
panel_interpolate = []
for i in tqdm(panel_input.id_post_placebo.unique()):
    df = panel_input[panel_input['id_post_placebo'] == i].sort_values('days_since_publication')
    df['approx_likes'] = df.likes.interpolate()
    df['approx_shares'] = df.shares.interpolate()
    df['approx_comments'] = df.comments.interpolate()
    df['approx_reactions'] = df.reactions.interpolate()
    df['approx_interactions'] = df.apply(lambda x: x['approx_likes'] 
                                         + x['approx_shares'] 
                                         + x['approx_comments'] 
                                         + x['approx_reactions'], axis = 1)
    panel_interpolate.append(df)   

100%|██████████| 81423/81423 [41:03<00:00, 33.05it/s] 


In [486]:
df_interpolate = pd.concat(panel_interpolate)

In [487]:
df_interpolate['growth_likes'] = df_interpolate.groupby(['id_post_placebo']).approx_likes.pct_change()
df_interpolate['growth_shares'] = df_interpolate.groupby(['id_post_placebo']).approx_shares.pct_change()
df_interpolate['growth_reactions'] = df_interpolate.groupby(['id_post_placebo']).approx_reactions.pct_change()
df_interpolate['growth_comments'] = df_interpolate.groupby(['id_post_placebo']).approx_comments.pct_change()
df_interpolate['growth_interactions'] = df_interpolate.groupby(['id_post_placebo']).approx_interactions.pct_change()

In [488]:
df_interpolate = df_interpolate.reset_index().drop('index', axis = 1)

In [489]:
pyreadr.write_rdata('/Users/cblanesg/misinformation_socialmedia/data/4-panel_data/misinformation/panel_interpolate_placebo.RData', df_interpolate, df_name="panel_placebo")

In [351]:
temp = posts_placebo.drop_duplicates('id_desinformacion', keep = 'first')
temp['id_placebo'] = temp['post_control'].apply(lambda x: str(uuid.uuid3(uuid.NAMESPACE_URL,x)))
temp[['id_desinformacion', 'id_placebo', 'date_desinformacion', 'link_control']].rename(columns = {'link_control':'link_placebo'})

,id_desinformacion,id_placebo,date_desinformacion,link_placebo
31537,ff5b98f7-7da4-3f5e-9b5b-c123c032defe,ea8777df-6af1-331d-a9e9-8ffae4a27171,2020-06-01,https://www.bluradio.com/economia/que-diferenc...


In [353]:
temp = scrapes_same_day[['id_desinformacion', 
                  'id_article', 
                 'date', 
                 'url_article']].rename(columns = {'id_article':'id_placebo', 
                                                                       'date':'date_desinformacion', 
                                                  'url_article':'link_placebo'})

,id_desinformacion,id_placebo,date_desinformacion,link_placebo
209,f159041b-2095-364f-8008-2e0379a8a579,d960d911-6b60-347b-afe9-5744109270c2,2019-07-22,https://bigtimenews.home.blog/2019/07/22/piche...


In [222]:


df_posts = df_posts[df_posts['id_desinformacion'].isin(id_placebo_media.id_desinformacion)]

In [223]:
df_posts = df_posts[df_posts['dummy_text_article'] == 1]
df_posts = df_posts[['id_desinformacion','link_control', 'date_control']].rename(columns = {'link_control':'link_placebo', 
                                                            'date_control':'date_placebo'})

In [224]:
placebo_id = pd.merge(left = id_placebo_media, 
        right = df_posts, 
        on = 'id_desinformacion')

In [225]:
placebo_id['date_desinformacion'] = placebo_id['date_desinformacion'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [226]:
placebo_id['filter'] = np.where(placebo_id['date_desinformacion'] == placebo_id['date_placebo'], 'same day', 'not same day')

In [227]:
placebo_id = placebo_id[placebo_id['filter'] == 'same day']

In [230]:
placebo_newspapers = placebo_id.drop_duplicates('id_desinformacion', keep = 'first')

In [236]:
missing_newspapers = id_placebo[~id_placebo['id_desinformacion'].isin(placebo_newspapers.id_desinformacion)]

In [244]:
missing_newspapers['date_desinformacion'] = missing_newspapers.apply(lambda x: datetime.date(x['year'], 
                                                                                         x['month'], 
                                                                                        x['days']), axis = 1)

In [248]:
path_scrapes = '/Users/cblanesg/misinformation_socialmedia/data/3-synthetic_controls/newspapers/1-scrapes/'

In [256]:
missing_newspapers= missing_newspapers.reset_index()

In [263]:
matches_scrapes = []
for i in range(0, len(missing_newspapers)):
    if missing_newspapers.loc[i]['media_outlet'] + '.json' in listdir(path_scrapes):
        scrapes = pd.read_json(path_scrapes + missing_newspapers.loc[i]['media_outlet'] + '.json')
        
        date_desinformation = missing_newspapers.loc[i]['date_desinformacion'].strftime('%Y-%m-%d')
        scrapes_same_day = scrapes[scrapes['date'] == date_desinformation]
        if len(scrapes_same_day) > 0:
            scrapes_same_day['id_desinformacion'] = missing_newspapers.loc[i]['id_desinformacion']
            scrapes_same_day = scrapes_same_day.drop_duplicates('id_desinformacion', keep = 'first')
            matches_scrapes.append(scrapes_same_day)
        else:
            pass

In [265]:
matches_scrapes = pd.concat(matches_scrapes)

In [275]:
len(id_newspapers) - 135

362

In [268]:
len(placebo_newspapers)

121

In [271]:
len(placebo_newspapers) + len(matches_scrapes)

135

In [273]:
placebo_newspapers

,Unnamed: 0,id_desinformacion,id_factcheck,link_desinformacion,label_desinformacion,media_outlet,link_medio,date_desinformacion,year,month,days,link_placebo,date_placebo,filter
78,4,032787bb-a1c6-300b-98de-418c660c7f45,f6be5f00-9051-3b67-aeba-0dd9039ccc6b,https://www.elespectador.com/noticias/el-mundo...,true,elespectador,https://www.elespectador.com,2019-02-21,2019,2,21,https://www.elespectador.com/deportes/futbol-c...,2019-02-21,same day
400,5,03f7bc37-b6be-3527-ad82-14b58db8c7a1,fa778a90-3f12-3b56-b430-b5420e259b6d,https://www.washingtonpost.com/politics/2020/0...,misleading,washingtonpost,https://www.washingtonpost.com,2020-02-26,2020,2,26,https://www.washingtonpost.com/weather/2021/03...,2020-02-26,same day
883,6,050bb5bc-45ee-30bd-be00-82b81dafbbd2,4b9018ed-3e84-3561-be8a-d65b890f82de,https://www.elespectador.com/noticias/nacional...,true,elespectador,https://www.elespectador.com,2019-07-26,2019,7,26,https://www.elespectador.com/deportes/futbol-c...,2019-07-26,same day
1262,16,07e35e56-ab97-35d4-9a60-4e1df7f4cdb6,f900b43e-9e6f-3fec-b268-40d5b56c7957,https://www.pulzo.com/nacion/alvaro-uribe-sugi...,true,pulzo,https://www.pulzo.com,2019-04-09,2019,4,9,https://www.pulzo.com/nacion/balacera-carrera-...,2019-04-09,same day
1662,17,089e8d7d-5a19-38e5-8e6a-a67faa11f964,f9f12565-527a-3470-b5c2-5b7824b162a5,https://www.lafm.com.co/colombia/vicepresident...,true,lafm,https://www.lafm.com.co,2019-02-12,2019,2,12,https://www.lafm.com.co/entretenimiento/yalitz...,2019-02-12,same day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68143,484,f9379c15-c5c4-3576-a3cb-f27a25c0c91c,ff2a784d-e3ef-3bd8-91ed-83d2de4ed7a1,https://seguimiento.co/la-region-caribe/alejan...,misleading,seguimiento,https://seguimiento.co,2020-02-04,2020,2,4,https://seguimiento.co/deportes/ciro-solano-es...,2020-02-04,same day
69207,491,fbcdfc1f-38b8-3ffb-9851-a9207129d49b,603f4a78-c013-354c-8717-158fcf1343b4,https://www.publimetro.co/co/noticias/2019/05/...,true,publimetro,https://www.publimetro.co,2019-05-06,2019,5,6,https://www.publimetro.co/co/noticias/2021/03/...,2019-05-06,same day
69701,492,fc1ab0a5-4099-33c9-8a5f-5a31d4c12833,b92197bc-2174-3e3c-b72c-7b053f29608d,https://www.elespectador.com/economia/contralo...,true,elespectador,https://www.elespectador.com,2020-10-25,2020,10,25,https://www.elespectador.com/deportes/futbol-c...,2020-10-25,same day
70143,493,fc2f6243-82d5-47b2-b092-ba1d0ca0a2c6,d793dd4e-a758-3d57-ac50-6024a548cfa0,https://www.ufo-spain.com/2019/01/07/filtracio...,misleading,ufo-spain,https://www.ufo-spain.com,2019-01-07,2019,1,7,https://www.ufo-spain.com/2021/03/10/fotografi...,2019-01-07,same day


In [242]:
pd.read_json('/Users/cblanesg/misinformation_socialmedia/data/3-synthetic_controls/newspapers/1-scrapes/' + 'argumentopolitico.json')




,id_article,url_article,newspaper,author,date,body,title
0,228f6ce1-1b28-3b9e-a5ce-2c1c2cf62a1a,http://www.argumentopolitico.com/2020/10/ejecu...,argumentopolitico,NaN,2020-10-01,FOTO DE INTERNET ANUNCIOS El regaetonero conoc...,\nEjecutan al famoso reguetonero “Rolex Style”...
1,7881194c-d0bc-3f62-acc8-67656201344a,http://www.argumentopolitico.com/2020/10/novio...,argumentopolitico,NaN,2020-10-01,FOTO DE INTERNET ANUNCIOSUna curiosa historia ...,\nNovios piden un Uber para ir a un hotel y el...
2,19bae1ff-e73c-3279-afc0-89707755dbfe,http://www.argumentopolitico.com/2020/10/sujet...,argumentopolitico,NaN,2020-10-01,FOTO DE INTERNET,\nSujeto que MAT0 a su perro por morder a a su...
3,2d2be1d7-141e-3cc6-95b6-ee921246bd80,http://www.argumentopolitico.com/2020/10/quien...,argumentopolitico,NaN,2020-10-01,FOTO DE INTERNET ANUNCIOS En diversos medios t...,"\n¿Quién es #LordEstafador, el sospecho del fe..."
4,3eaabfd4-07d8-394d-b5cc-5819186cc06d,http://www.argumentopolitico.com/2020/10/nina-...,argumentopolitico,NaN,2020-10-01,FOTO DE INTERNET ANUNCIOS Un niña logró huir d...,\nNiña escapa de sus secuestradores y logran d...
...,...,...,...,...,...,...,...
7527,2edcd04b-2db5-3d7b-b552-3dc806e1b932,http://www.argumentopolitico.com/2016/03/cuand...,argumentopolitico,NaN,2016-03-01,AnunciosLos seguros de automóviles nos ofrece...,\n¿Cuándo se invalida mi seguro de automóvil?\n
7528,43ecfd57-b9f7-3edf-882f-97c04ab2b90f,http://www.argumentopolitico.com/2016/02/const...,argumentopolitico,NaN,2016-02-01,AnunciosSer constante a la hora de ahorrar es ...,"\nConstancia al ahorrar, es el secreto para fo..."
7529,53dc70ba-4426-3761-af7f-b1682f1e5d02,http://www.argumentopolitico.com/2016/02/por-f...,argumentopolitico,NaN,2016-02-01,El Poder Judicial de la Federación (PJF) se ...,\nAusteridad en el Poder Judicial; ahorrará 43...
7530,b60e96c2-7719-37ea-925f-686b639fab59,http://www.argumentopolitico.com/2016/02/mexic...,argumentopolitico,NaN,2016-02-01,"CDMX. Joe Biden, vicepresidente de Estados Uni...",\nMéxico lleno de violencia e impunidad: Biden\n


In [123]:
placebo_id[['link_desinformacion', 
            'link_placebo', 
            'date_desinformacion', 
           'date_placebo']]

,link_desinformacion,link_placebo,date_desinformacion,date_placebo
78,https://www.elespectador.com/noticias/el-mundo...,https://www.elespectador.com/deportes/futbol-c...,2019-02-21,2019-02-21
80,https://www.elespectador.com/noticias/el-mundo...,https://www.elespectador.com/deportes/futbol-c...,2019-02-21,2019-02-21
81,https://www.elespectador.com/noticias/el-mundo...,https://www.elespectador.com/deportes/futbol-c...,2019-02-21,2019-02-21
82,https://www.elespectador.com/noticias/el-mundo...,https://www.elespectador.com/deportes/futbol-c...,2019-02-21,2019-02-21
83,https://www.elespectador.com/noticias/el-mundo...,https://www.elespectador.com/deportes/futbol-c...,2019-02-21,2019-02-21
...,...,...,...,...
70639,https://www.bluradio.com/salud/celebre-medico-...,https://www.bluradio.com/economia/que-diferenc...,2020-06-01,2020-06-01
70640,https://www.bluradio.com/salud/celebre-medico-...,https://www.bluradio.com/economia/que-diferenc...,2020-06-01,2020-06-01
70641,https://www.bluradio.com/salud/celebre-medico-...,https://www.bluradio.com/economia/que-diferenc...,2020-06-01,2020-06-01
70642,https://www.bluradio.com/salud/celebre-medico-...,https://www.bluradio.com/economia/que-diferenc...,2020-06-01,2020-06-01


## Placebo Social

In [176]:
posts_social = []
for i in listdir('../../../data/3-synthetic_controls/social/5-clean_controls/'):
    try:
        posts_social.append(pd.read_json('../../../data/3-synthetic_controls/social/5-clean_controls/' + i))
    except:
        continue

In [177]:
posts_social = pd.concat(posts_social)

In [189]:
posts_social_match = posts_social[posts_social['id_desinformacion'].isin(list(panel_reg.id_desinformacion))]

In [190]:
len(posts_social_match)

20846

In [204]:
posts_social_match['filter_history'] = np.where(posts_social_match['history_control'] == None, 1, 0)

In [207]:
posts_social_match = posts_social_match[~posts_social_match['history_control'].isnull()]

In [208]:
len(posts_social)

58706

In [209]:
same_day = posts_social_match[posts_social_match['n_days_since_desinformation'] == 0]
post_social_placebo = same_day[same_day['n_days_since_desinformation'] == 0].drop_duplicates('id_desinformacion', keep = 'first')[['id_desinformacion', 
                                                                                                                    'link_user_desinformacion', 
                                                                                                                                           'link_post',
                                                                                                                    'date_desinformacion', 
                                                                                                                    'date_post', 
                                                                                                                    'history_control']].rename(columns = {'date_post':'date_placebo', 
                                                                                                                                                   'link_post':'link_placebo', 
                                                                                                                                                   'history_control':'history_placebo'})

In [210]:
post_social_placebo['id_placebo'] = post_social_placebo['link_placebo'].apply(lambda x: str(uuid.uuid3(uuid.NAMESPACE_URL,x)))

In [211]:
post_social_placebo = post_social_placebo.reset_index().drop('index', axis = 1)

In [212]:
post_social_placebo = post_social_placebo[['id_desinformacion', 
                    'id_placebo', 
                    'link_user_desinformacion', 
                    'link_placebo', 
                    'date_desinformacion', 
                    'date_placebo', 
                    'history_placebo']]

In [213]:
social_misinformation = panel_reg[panel_reg['type_misinformation'] == 'social_media']
missing_placebo_social = social_misinformation[~social_misinformation['id_desinformacion'].isin(post_social_placebo.id_desinformacion)]

In [214]:
len(missing_placebo_social)

53

In [217]:
len(post_social_placebo)

56

In [218]:
post_social_placebo

,id_desinformacion,id_placebo,link_user_desinformacion,link_placebo,date_desinformacion,date_placebo,history_placebo
0,cedadf18-8675-3758-b3f8-2fbb375dcfd4,473ed18e-6553-3c89-9870-1f6012c29e0d,https://www.facebook.com/104894350990942,https://www.facebook.com/104894350990942/posts...,2020-04-30 16:04,2020-04-30,"[{'actual': {'likeCount': 38, 'shareCount': 17..."
1,72da339b-3830-4289-bb72-77db3e488d79,878b703c-9cf3-3c33-9e3e-c9066c9a5573,https://www.facebook.com/HayQueContarlo,https://www.facebook.com/HayQueContarlo/posts/...,2020-03-30 01:03,2020-03-30,"[{'actual': {'likeCount': 52, 'shareCount': 61..."
2,117a5f43-7cba-4b7d-b6e4-d0141baeeba4,52374bf6-4f70-3b12-9e4f-d50b4911f2e8,https://www.facebook.com/RadioKawsachunCoca,https://www.facebook.com/RadioKawsachunCoca/po...,2020-06-21 12:06,2020-06-21,"[{'actual': {'likeCount': 672, 'shareCount': 8..."
3,67278894-02b5-11eb-87de-acde48001122,9823b662-c6ee-31ab-8bf0-c1749238d564,https://www.facebook.com/HagamosPatriaInteligente,https://www.facebook.com/HagamosPatriaIntelige...,2019-08-13 15:08,2019-08-13,"[{'actual': {'likeCount': 111, 'shareCount': 2..."
4,6881ef69-7cde-3cf4-8b75-4a1909e9ef7f,f45a4be3-fb55-389d-9878-15b9e6917932,https://www.facebook.com/PorUnPeruSinCorruptoss,https://www.facebook.com/PorUnPeruSinCorruptos...,2019-12-28 22:12,2019-12-28,"[{'actual': {'likeCount': 251, 'shareCount': 2..."
5,ce5417e0-0b86-3674-ba78-1a2ab5957eef,6e03efaf-c381-395a-bec2-ace9945fd692,https://www.facebook.com/274870826446524,https://www.facebook.com/274870826446524/posts...,2020-01-30 18:01,2020-01-30,"[{'actual': {'likeCount': 0, 'shareCount': 0, ..."
6,15b5fd51-ca22-4ed4-971b-2549ca8f95d9,d3280acb-5c94-301c-bf25-bf67a4af0c6c,https://www.facebook.com/1000manerasdeanulartu...,https://www.facebook.com/1000manerasdeanulartu...,2020-07-14 16:07,2020-07-14,"[{'actual': {'likeCount': 121, 'shareCount': 9..."
7,7292e953-2497-43ae-b136-c07cfd325acd,fe5d3d9a-0df2-39c4-a1c7-e264ae117e02,https://www.facebook.com/RadioConcordiaDigital,https://www.facebook.com/RadioConcordiaDigital...,2020-04-02 18:04,2020-04-02,"[{'actual': {'likeCount': 80, 'shareCount': 41..."
8,672787ea-02b5-11eb-87de-acde48001122,ec2d4a50-b804-3ca6-b1ee-f37b4dda5696,https://www.facebook.com/OLDCUACK,https://www.facebook.com/OLDCUACK/posts/806321...,2019-08-22 20:08,2019-08-22,"[{'actual': {'likeCount': 9, 'shareCount': 6, ..."
9,3f7e5015-4139-3e57-a44f-02c4ea96677a,c8c87249-4edd-3356-9a25-b9ec13ae843f,https://www.facebook.com/571680776577702,https://www.facebook.com/571680776577702/posts...,2020-03-28 16:03,2020-03-28,"[{'actual': {'likeCount': 1, 'shareCount': 1, ..."
